In [12]:
! pip install pandas

In [13]:
# imports

import pandas as pd

import os


# Get the current directory
current_dir = os.getcwd()

In [14]:
test = True

In [23]:
current_dir = os.getcwd()

if test:
    print('current_dir: ' + str(current_dir))

#csv_path = current_dir + "\Top_25_World_News_2018_2023\WorldNewsData.csv"
csv_path = r"{}\Top_25_World_News_2018_2023\WorldNewsData.csv".format(current_dir)

if test:
    print('csv_path: ' + str(csv_path))

# convert csv to pandas dataframe
data = pd.read_csv(csv_path)

if test:
    print('len(data)    :' + str(len(data))    + '\n')
    print('data.iloc[0] :' + str(data.iloc[0]) + '\n')
    print('data.head()  :' + str(data.head())  + '\n')

current_dir: c:\Users\vanvi\Desktop\workspace\cmpe-256-project
csv_path: c:\Users\vanvi\Desktop\workspace\cmpe-256-project\Top_25_World_News_2018_2023\WorldNewsData.csv
len(data)    :1859
data.iloc[0] :Date                                          May 01, 2018
Top1     North Korea to open its sky, South Korean medi...
Top2           The Mueller probe ain't ending anytime soon
Top3     BRAND NEW: 2018 – Renault Alpine A110 – Start-...
Top4     The story of the Midwestern innocent let loose...
Top5     Mattis tempers optimistic outlook on Afghanist...
Top6     Facebook is taking on Tinder with new dating f...
Top7     OECD chief says Greece deserves debt relief af...
Top8                     Facebook to launch dating service
Top9     U.N. could help Myanmar gather evidence of cri...
Top10    PTSD veterans 'cured' with ecstasy treatment i...
Top11    May Day protesters demand better rights for wo...
Top12    China’s Social Credit System Punishes Low-Scor...
Top13             Paris police 